In [19]:
import os

downloads_path = os.path.expanduser("~/Downloads")
os.chdir(downloads_path)

In [20]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

hvacdata = pd.read_csv('hvac_merged.csv')


/var/folders/9f/3yz89nb525nczbywh5dl11kh0000gn/T/ipykernel_32452/2688843383.py:6: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  hvacdata = pd.read_csv('hvac_merged.csv')


In [21]:
print(hvacdata.columns.tolist())



# show up to 200 rows and all columns
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)


print(hvacdata.head(50))



['uic', 'job_seq', 'apl', 'cause_code', 'cdm_rin', 'csmp_narrative_summary', 'date_closing', 'date_maintenance_action', 'deferral_reason_code', 'due_date', 'eic', 'equipment_nomenclature', 'eswbs_opening', 'iuc_screening_code', 'location', 'mhc', 'mho', 'mhr', 'priority_code', 'safety_code', 'status_code', 'tycom_screening_code', 'type_availability_code', 'type_of_maintenance_action', 'when_discovered_code', 'work_center', 'total_ima_man_hours', 'total_ship_force_man_hours', 'total_replacement_cost', 'total_repair_replacement_cost', 'def_narrative', 'nondef_narrative', 'closing_narrative', 'ima_narrative', 'total_material_cost', 'opening_ship_system', 'job_status', 'days_open', 'action_taken', 'intermediate_unit_commander_screening', 'priority', 'type_of_availability_needed', 'tycom_screening', 'when_discovered', 'cause', 'deferral_reason', 'feasible_with_ship_inventory', 'feasible_with_additional_dla_inventory', 'ship', 'uic_s', 'jsn', 'date_closing_s', 'demand_date', 'issue_date', 'c

In [22]:


hvacdata['date_maintenance_action'] = pd.to_datetime(hvacdata['date_maintenance_action'], errors='coerce')

# group by `ship` to find the earliest date
rcoh_start_dates = (
    hvacdata.groupby('ship')['date_maintenance_action']
    .min()
    .reset_index()
    .rename(columns={'date_maintenance_action': 'rcoh_start_date'})
)

print(rcoh_start_dates)



# !!!!! RCOH for CVN 68 took place between 1998 and 2001, but the first maintenace job recorded in the dataset for CVN 68 is in 2017?
# Just use start dates within the data? This may not accurately represent the true project phases

     ship rcoh_start_date
0   CVN68      2017-11-02
1   CVN69      2017-11-01
2   CVN70      2017-11-01
3   CVN71      2017-11-04
4   CVN72      2017-11-01
5   CVN73      2017-11-02
6   CVN74      2017-11-01
7   CVN75      2017-11-02
8   CVN76      2017-11-01
9   CVN77      2017-11-01
10  CVN78      2017-11-01


In [23]:
hvacdata = hvacdata.merge(rcoh_start_dates, on='ship', how='left')


In [24]:
# calculate time since RCOH start

hvacdata['months_since_rcoh_start'] = (
    (hvacdata['date_maintenance_action'] - hvacdata['rcoh_start_date']) / np.timedelta64(1, 'M')
).astype(int)
